
# SQL Queries for Machine Learning

In this notebook, I illustrate how to perform complex SQL queries to:

- Create tables and populate them with sample data pertinent to the CECL model.
- Calculate key metrics, such as loan default rates and rolling default rates, which are critical for understanding credit risk.
- Preprocess data for building a ML model.


**Step 1: Set up SQLite3**


In [1]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect(':memory:')  # Use in-memory database for demonstration
cursor = conn.cursor()


**Assume we have the following tables in our database:**

- **loans**: This table contains information about individual loans, including details such as loan amount, interest rate, term, and status.

- **payments**: This table holds information about loan payments, including payment dates, amounts, and associated loan IDs.

- **borrowers**: This table includes information about borrowers, such as their names, contact details, credit scores, and demographic information.

- **economic_indicators**: This table contains economic data that may influence loan performance, including metrics like unemployment rates, inflation rates, and GDP growth.




**Step 2: Create the tables**

In this step, we will create the tables to store information about loans, payments, borrowers, and economic indicators using the following SQL commands:

In [2]:
# Create tables
cursor.execute('''
CREATE TABLE loans (
    loan_id INTEGER PRIMARY KEY,
    borrower_id INTEGER,
    loan_amount REAL,
    loan_date DATE,
    maturity_date DATE,
    interest_rate REAL,
    loan_status TEXT
);
''')

cursor.execute('''
CREATE TABLE payments (
    payment_id INTEGER PRIMARY KEY,
    loan_id INTEGER,
    payment_date DATE,
    payment_amount REAL,
    payment_status TEXT
);
''')

cursor.execute('''
CREATE TABLE borrowers (
    borrower_id INTEGER PRIMARY KEY,
    name TEXT,
    credit_score INTEGER,
    income REAL,
    employment_status TEXT
);
''')

cursor.execute('''
CREATE TABLE economic_indicators (
    date DATE,
    unemployment_rate REAL,
    gdp_growth REAL,
    interest_rate REAL
);
''')


**Step 2: Insert data into tables:**

In [3]:
# Insert data into loans
cursor.executemany('''
INSERT INTO loans (loan_id, borrower_id, loan_amount, loan_date, maturity_date, interest_rate, loan_status) VALUES (?, ?, ?, ?, ?, ?, ?);
''', [
    (1, 1, 100000, '2020-01-01', '2025-01-01', 0.05, 'Active'),
    (2, 2, 150000, '2020-02-01', '2025-02-01', 0.06, 'Defaulted'),
    (3, 3, 200000, '2020-03-01', '2025-03-01', 0.04, 'Active')
])

# Insert data into payments
cursor.executemany('''
INSERT INTO payments (payment_id, loan_id, payment_date, payment_amount, payment_status) VALUES (?, ?, ?, ?, ?);
''', [
    (1, 1, '2020-02-01', 500, 'Paid'),
    (2, 1, '2020-03-01', 500, 'Paid'),
    (3, 2, '2020-02-01', 750, 'Defaulted'),
    (4, 3, '2020-03-01', 1000, 'Paid')
])

# Insert data into borrowers
cursor.executemany('''
INSERT INTO borrowers (borrower_id, name, credit_score, income, employment_status) VALUES (?, ?, ?, ?, ?);
''', [
    (1, 'John Doe', 750, 80000, 'Employed'),
    (2, 'Jane Smith', 600, 60000, 'Unemployed'),
    (3, 'Alice Johnson', 800, 90000, 'Employed')
])

# Insert data into economic_indicators
cursor.executemany('''
INSERT INTO economic_indicators (date, unemployment_rate, gdp_growth, interest_rate) VALUES (?, ?, ?, ?);
''', [
    ('2020-01-01', 5.0, 2.5, 0.03),
    ('2020-02-01', 5.2, 2.4, 0.03),
    ('2020-03-01', 5.5, 2.3, 0.03)
])


*italicized text*
## Complex SQL Queries:

**Step 4: Calculate Loan Default Rates**



In [7]:
# Calculate Loan Default Rates
query = '''
SELECT
    l.loan_id,
    l.borrower_id,
    l.loan_amount,
    l.loan_date,
    l.maturity_date,
    l.interest_rate,
    l.loan_status,
    b.name,
    b.credit_score,
    b.income,
    b.employment_status,
    COUNT(p.payment_id) AS total_payments,
    SUM(CASE WHEN p.payment_status = 'Defaulted' THEN 1 ELSE 0 END) AS defaulted_payments,
    (SUM(CASE WHEN p.payment_status = 'Defaulted' THEN 1 ELSE 0 END) * 1.0 / NULLIF(COUNT(p.payment_id), 0)) AS default_rate
FROM
    loans l
LEFT JOIN
    payments p ON l.loan_id = p.loan_id
JOIN
    borrowers b ON l.borrower_id = b.borrower_id
GROUP BY
    l.loan_id,
    l.borrower_id,
    l.loan_amount,
    l.loan_date,
    l.maturity_date,
    l.interest_rate,
    l.loan_status,
    b.name,
    b.credit_score,
    b.income,
    b.employment_status
ORDER BY
    l.loan_id;
'''

# Execute the query
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Display the results
import pandas as pd

# Create a DataFrame for better visualization
columns = [
    'Loan ID', 'Borrower ID', 'Loan Amount', 'Loan Date', 'Maturity Date',
    'Interest Rate', 'Loan Status', 'Name', 'Credit Score', 'Income',
    'Employment Status', 'Total Payments', 'Defaulted Payments', 'Default Rate'
]

df_default_rates = pd.DataFrame(results, columns=columns)
print(df_default_rates)


   Loan ID  Borrower ID  Loan Amount   Loan Date Maturity Date  Interest Rate  \
0        1            1     100000.0  2020-01-01    2025-01-01           0.05   
1        2            2     150000.0  2020-02-01    2025-02-01           0.06   
2        3            3     200000.0  2020-03-01    2025-03-01           0.04   

  Loan Status           Name  Credit Score   Income Employment Status  \
0      Active       John Doe           750  80000.0          Employed   
1   Defaulted     Jane Smith           600  60000.0        Unemployed   
2      Active  Alice Johnson           800  90000.0          Employed   

   Total Payments  Defaulted Payments  Default Rate  
0               2                   0           0.0  
1               1                   1           1.0  
2               1                   0           0.0  


**Step 5: Calculate Rolling Default Rates**

In [8]:
# Calculate Rolling Default Rates
rolling_query = '''
WITH loan_defaults AS (
    SELECT
        l.loan_id,
        l.borrower_id,
        l.loan_amount,
        l.loan_date,
        l.maturity_date,
        l.interest_rate,
        l.loan_status,
        b.name,
        b.credit_score,
        b.income,
        b.employment_status,
        p.payment_date,
        p.payment_status,
        ROW_NUMBER() OVER (PARTITION BY l.loan_id ORDER BY p.payment_date) AS payment_seq
    FROM
        loans l
    LEFT JOIN
        payments p ON l.loan_id = p.loan_id
    JOIN
        borrowers b ON l.borrower_id = b.borrower_id
)
SELECT
    loan_id,
    borrower_id,
    loan_amount,
    loan_date,
    maturity_date,
    interest_rate,
    loan_status,
    name,
    credit_score,
    income,
    employment_status,
    payment_date,
    payment_status,
    SUM(CASE WHEN payment_status = 'Defaulted' THEN 1 ELSE 0 END) OVER (PARTITION BY loan_id ORDER BY payment_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_defaults,
    COUNT(*) OVER (PARTITION BY loan_id ORDER BY payment_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS rolling_payments,
    (SUM(CASE WHEN payment_status = 'Defaulted' THEN 1 ELSE 0 END) OVER (PARTITION BY loan_id ORDER BY payment_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) * 1.0 /
     NULLIF(COUNT(*) OVER (PARTITION BY loan_id ORDER BY payment_date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 0)) AS rolling_default_rate
FROM
    loan_defaults
ORDER BY
    loan_id,
    payment_date;
'''

# Execute the rolling default rates query
cursor.execute(rolling_query)

# Fetch the results
rolling_results = cursor.fetchall()

# Create a DataFrame for rolling default rates
columns_rolling = [
    'Loan ID', 'Borrower ID', 'Loan Amount', 'Loan Date', 'Maturity Date',
    'Interest Rate', 'Loan Status', 'Name', 'Credit Score', 'Income',
    'Employment Status', 'Payment Date', 'Payment Status', 'Rolling Defaults',
    'Rolling Payments', 'Rolling Default Rate'
]

df_rolling_default_rates = pd.DataFrame(rolling_results, columns=columns_rolling)
print(df_rolling_default_rates)


   Loan ID  Borrower ID  Loan Amount   Loan Date Maturity Date  Interest Rate  \
0        1            1     100000.0  2020-01-01    2025-01-01           0.05   
1        1            1     100000.0  2020-01-01    2025-01-01           0.05   
2        2            2     150000.0  2020-02-01    2025-02-01           0.06   
3        3            3     200000.0  2020-03-01    2025-03-01           0.04   

  Loan Status           Name  Credit Score   Income Employment Status  \
0      Active       John Doe           750  80000.0          Employed   
1      Active       John Doe           750  80000.0          Employed   
2   Defaulted     Jane Smith           600  60000.0        Unemployed   
3      Active  Alice Johnson           800  90000.0          Employed   

  Payment Date Payment Status  Rolling Defaults  Rolling Payments  \
0   2020-02-01           Paid                 0                 1   
1   2020-03-01           Paid                 0                 2   
2   2020-02-01      D

**Step 6: Close the connection**

In [9]:

conn.close()




# Building a CECL model using our Data:



**Step 1: Feature Engineering**

In this step, we will focus on the following features for the CECL model:

- Loan amount
- Borrower credit score
- Rolling default rates
- Economic indicators


In [11]:
# Feature Engineering
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Create a new DataFrame for modeling
model_data = df_rolling_default_rates.copy()

# Create a binary target variable for default (1 if defaulted, 0 otherwise)
model_data['defaulted'] = model_data['Payment Status'].apply(lambda x: 1 if x == 'Defaulted' else 0)

# Select relevant features for the model
features = [
    'Loan Amount', 'Credit Score', 'Rolling Default Rate', 'Income', 'Interest Rate'
]

# Prepare the feature set and target variable
X = model_data[features]
y = model_data['defaulted']


**Step 2: Data Preparation**

train_test split

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print(f'Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}')


Training set shape: (3, 5), Testing set shape: (1, 5)



**Step 3: Model Training**

using a logistic regression model for binary classification to predict loan defaults


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate predicted probabilities (Probability of Default)
y_prob = model.predict_proba(X_test)[:, 1]


**Step 4: Model Evaluation**

to evaluate the model's performance using classification metrics

In [25]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[1]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(



## Conclusion

In this notebook, I illustrated how to execute complex SQL queries to build a Current Expected Credit Loss (CECL) model.

### Main SQL Queries Used:

1. **Creating Tables and Input Data**:
   - The query establishes the necessary database structure for storing essential information related to loans, payments, and borrowers. It ensures that the data is organized and accessible for subsequent analysis.

2. **Loan Default Rates Calculation**:
   - This query calculates the default rates for individual loans by joining the `loans`, `payments`, and `borrowers` tables. It retrieves key details about each loan and borrower, counts the total number of payments made, and sums the number of defaulted payments to compute the default rate. The results are presented in a pandas DataFrame, facilitating easy visualization and enhancing the understanding of loan performance metrics.

3. **Rolling Default Rates Calculation**:
   - This query employs a Common Table Expression (CTE) to analyze rolling default rates based on the last three payments for each loan. It collects loan and payment details, assigning a sequence number to each payment for rolling calculations. The query computes the number of rolling defaults and total payments, deriving the rolling default rate over the specified period. The results are also displayed in a pandas DataFrame, providing valuable insights into the trends of loan defaults over time.

-